In [3]:
import pandas as pd 

In [10]:
import spacy

In [11]:
nlp = spacy.load('en_core_web_lg')

In [4]:
data_df = pd.read_csv('FC_large.csv')

/Users/spangher/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: DtypeWarning: Columns (0,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [27]:
texts = data_df['ScrapedText'].fillna('').tolist()

In [49]:
from tqdm.auto import tqdm

In [50]:
try:
    nlp.add_pipe('sentencizer')
except:
    pass 

all_sents = []
for doc in tqdm(nlp.pipe(texts, disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer", "ner"]), total=len(data_df)):
    all_sents.append(list(map(lambda x: str(x).strip(), doc.sents)))

  0%|          | 0/32416 [00:00<?, ?it/s]

In [ ]:
data_df['sentence'] = all_sents

In [65]:
import jsonlines

In [84]:
output_jsonl = data_df[['url', 'sentence']].loc[lambda df: df['sentence'].str.len() > 0].rename(columns={'url': 'doc_idx'}).to_dict(orient='records')

In [86]:
with open('matt-data.jsonl', 'w') as f:
    jsonlines.Writer(f).write_all(output_jsonl)

# Run Rules Baselines

In [71]:
import sys
sys.path.insert(0, '../scripts/')
import util

In [ ]:
method_1_sents = []

for doc in tqdm(output_jsonl):
    _, output_sents = util.get_quotes_method_1(doc['sentence'], orig_speaking=False, return_sents=True)
    method_1_sents.append(output_sents)

  0%|          | 0/29311 [00:00<?, ?it/s]

In [119]:
method_1_output = []
for s, i in zip(method_1_sents, output_jsonl):
    method_1_output.append({
        'doc_idx': i['doc_idx'], 
        'sents_and_labels': list(map(lambda x: {'sent': str(x['sent']), 'sources': x['sources']}, s))
    })

In [120]:
with open('matts-processed-data-method-1.jsonl', 'w') as f:
    jsonlines.Writer(f).write_all(method_1_output)

In [100]:
method_2_sents = []

for doc in tqdm(output_jsonl):
    _, output_sents = util.get_quotes_method_2(doc['sentence'], orig_speaking=False, return_sents=True)
    method_2_sents.append(output_sents)

  0%|          | 0/29311 [00:00<?, ?it/s]

In [104]:
len(output_jsonl)

29311

In [105]:
len(method_2_sents)

29311

In [116]:
method_2_output = []
for s, i in zip(method_2_sents, output_jsonl):
    method_2_output.append({
        'doc_idx': i['doc_idx'], 
        'sents_and_labels': list(map(lambda x: {'sent': str(x['sent']), 'sources': x['sources']}, s))
    })

In [112]:
import json

In [117]:
for m in method_2_output:
    json.dumps(m)

In [118]:
with open('matts-processed-data-method-2.jsonl', 'w') as f:
    jsonlines.Writer(f).write_all(method_2_output)